# Install dependencies

In [21]:
!pip3 install pretty_midi bokeh mlflow

    100% |████████████████████████████████| 47.7MB 27kB/s 
    100% |████████████████████████████████| 92kB 12.8MB/s 
    100% |████████████████████████████████| 81kB 11.2MB/s 
    100% |████████████████████████████████| 122kB 11.0MB/s 
    100% |████████████████████████████████| 1.0MB 1.5MB/s 
    100% |████████████████████████████████| 92kB 11.4MB/s 
    100% |████████████████████████████████| 51kB 11.9MB/s 
    100% |████████████████████████████████| 460kB 3.0MB/s 
  Running setup.py bdist_wheel for simplejson ... done
  Stored in directory: /home/jupyter/.cache/pip/wheels/5d/1a/1e/0350bb3df3e74215cd91325344cc86c2c691f5306eb4d22c77
  Running setup.py bdist_wheel for querystring-parser ... done
  Stored in directory: /home/jupyter/.cache/pip/wheels/ee/09/99/bf937e4f02788fa8b33dc5240842ba3977ba5c3c4ad4a115d7
  Running setup.py bdist_wheel for alembic ... done
  Stored in directory: /home/jupyter/.cache/pip/wheels/8b/65/b2/9837b4422d13e739c3324c428f1b3aa9e3c3df666bb420e4b3
  Running se

# Fetch and clean data

In [23]:
import pandas as pd
from tqdm import tqdm_notebook
import pretty_midi
import os
import shutil

In [1]:
!wget -P ../data https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip

--2019-06-30 12:37:23--  https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.166.128, 2a00:1450:400c:c0a::80
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.166.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59243107 (56M) [application/zip]
Saving to: ‘../data/maestro-v2.0.0-midi.zip’

maestro-v2.0.0-midi 100%[===================>]  56.50M  64.9MB/s    in 0.9s    

2019-06-30 12:37:25 (64.9 MB/s) - ‘../data/maestro-v2.0.0-midi.zip’ saved [59243107/59243107]



In [ ]:
!unzip ../data/maestro-v2.0.0-midi.zip -d ../data

In [5]:
metadata = pd.read_csv('../data/maestro-v2.0.0/maestro-v2.0.0.csv')

In [6]:
len(metadata)

1282

In [14]:
songs = []
for _, row in tqdm_notebook(list(metadata.iterrows())):
    midi = pretty_midi.PrettyMIDI(os.path.join('../data/maestro-v2.0.0', row['midi_filename']))
    midi.split = row['split']
    songs.append(midi)

In [18]:
shutil.rmtree('../data/songs.txt', ignore_errors=True)
with open('../data/songs.txt', 'w') as f:
    for midi in tqdm_notebook(songs):
        assert len(midi.instruments) == 1
        f.write('{split}#{tempo}#{notes}\n'.format(
            split=midi.split, tempo=midi.estimate_tempo(), 
            notes=[(n.start, n.end, n.pitch, n.velocity) for n in midi.instruments[0].notes]
        ))